In [ ]:
using MKL
using PhysicalUnits, LinearAlgebra
using CairoMakie
CairoMakie.activate!()
using MoireIVC.LLHF
using MoireIVC.LLHF_Plot

In [ ]:
N1 = 18; N2 = 18
num_para = LLHF_init_with_lambda(1.0; N1 = N1, N2 = N2);
LLHF_change_lambda!(num_para, 0.0);
num_para.H0 .= 0.05 * [num_para.system.W0 * sqrt(27/16) *
    (sin(-2π*k2/N2)+sin(-2π*k1/N1)+sin(2π*(k1/N1+k2/N2)))*(1.5-τn)*(τn′==τn) 
        for k1 in 0:N1-1, k2 in 0:N2-1, τn′ in 1:2, τn in 1:2
];

In [ ]:
symmetry = [LLHF.Rot3(0); LLHF.PT(0,:T)]
ρ = LLHF_solve(num_para; coherence = 0.0, 
    error_tolerance = 1E-10, max_iter_times = 100, 
    post_process_times = 100, post_procession = symmetry,
    complusive_mixing=false, complusive_mixing_rate=0.5, 
    stepwise_output = true, final_output = true
);

In [ ]:
LLHF_plot_band_3D(ρ; para=num_para)

In [ ]:
LLHF_plot_band(ρ; para=num_para)

In [ ]:
LLHF_plot_phase(ρ; para=num_para)

In [ ]:
LLHF_plot_Sz(ρ; para=num_para)

In [ ]:
LLHF_plot_Berrycurvature(ρ; para=num_para)

In [ ]:
LLHF_plot_realspace(ρ; para=num_para,
    N = 18, xlimits = (-1.2, 1.2), ylimits = (-1.2, 1.2), 
    arrowscale = 1, arrowsize = 4
)